In [42]:
import os, time, csv, glob, shutil
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, ElementNotInteractableException

import pandas as pd

# Path to the folder where CSVs should be cleared
CLEAR_DIR = r"C:\Users\Administrator\Rahul\downloads\downloads"

# Clear only .csv files in the folder
if os.path.exists(CLEAR_DIR):
    for filename in os.listdir(CLEAR_DIR):
        file_path = os.path.join(CLEAR_DIR, filename)
        if os.path.isfile(file_path) and filename.lower().endswith(".csv"):
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
else:
    print(f"Directory not found: {CLEAR_DIR}")

# ---------------- CONFIG ----------------
DOWNLOAD_DIR = os.path.abspath("./downloads")  # downloaded file target
BASE_URL = "https://appv2.loveinstore.com/Home"
MAX_WAIT_SECS = 45
HEADLESS = True

# Block size in days (set to 8 if you prefer)
BLOCK_DAYS = 10

# ✅ --- PUT YOUR LOGIN CREDENTIALS HERE ---
USERNAME = "MISDBRPharma-01"
PASSWORD = "Mis@LIS@2024"
# -----------------------------------------

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# =========================
# CHROME (headless) + downloads
# =========================
chrome_options = Options()
if HEADLESS:
    chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--ignore-ssl-errors=yes")

prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, MAX_WAIT_SECS)
actions = ActionChains(driver)

# =========================
# HELPERS (stale/interactable safe)
# =========================
def js_click(el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(locator, timeout=40, attempts=3, scroll=True):
    last_err = None
    for _ in range(attempts):
        try:
            el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))
            if scroll:
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                except Exception:
                    pass
            try:
                el.click()
            except Exception:
                js_click(el)
            return el
        except (StaleElementReferenceException, ElementNotInteractableException) as e:
            last_err = e
            time.sleep(0.3)
            continue
        except Exception as e:
            last_err = e
            time.sleep(0.3)
            continue
    raise last_err if last_err else TimeoutException(f"Could not click {locator}")

def switch_into_frame_containing(locator, timeout=30):
    driver.switch_to.default_content()
    try:
        WebDriverWait(driver, 2).until(EC.presence_of_element_located(locator))
        return False
    except TimeoutException:
        pass
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    for fr in frames:
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame(fr)
            WebDriverWait(driver, 2).until(EC.presence_of_element_located(locator))
            return True
        except Exception:
            continue
    driver.switch_to.default_content()
    return False

def ensure_in_correct_frame():
    anchor_locators = [
        (By.XPATH, "//*[@id='Sdatewise' or @name='Sdatewise' or @id='select2-Sdatewise-container']"),
        (By.ID, "Sdate"),
        (By.ID, "dateto"),
    ]
    for loc in anchor_locators:
        if switch_into_frame_containing(loc, timeout=5):
            return
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located(loc))
            return
        except Exception:
            continue

def find_first(locators, timeout_each=15):
    last_err = None
    for loc in locators:
        try:
            return WebDriverWait(driver, timeout_each).until(EC.visibility_of_element_located(loc))
        except Exception as e:
            last_err = e
    raise last_err

def wait_for_download(dir_path, timeout=240):
    start = time.time()
    seen = set(glob.glob(os.path.join(dir_path, "*")))
    while time.time() - start < timeout:
        time.sleep(1)
        current = set(glob.glob(os.path.join(dir_path, "*")))
        new_files = list(current - seen)
        if new_files:
            while any(p.endswith(".crdownload") for p in new_files):
                time.sleep(1)
                current = set(glob.glob(os.path.join(dir_path, "*")))
                new_files = list(current - seen)
            return max(new_files, key=os.path.getmtime)
    return None

def parse_table(file_path):
    lower = file_path.lower()
    try:
        if lower.endswith(".csv"):
            try:
                return pd.read_csv(file_path, encoding="utf-8")
            except Exception:
                return pd.read_csv(file_path, encoding="utf-8-sig")
        elif lower.endswith(".xlsx") or lower.endswith(".xls"):
            return pd.read_excel(file_path)
        else:
            return pd.read_csv(file_path, engine="python")
    except Exception:
        return None

def close_overlays():
    try:
        actions.send_keys(Keys.ESCAPE).perform(); time.sleep(0.1)
        actions.send_keys(Keys.ESCAPE).perform()
    except Exception:
        pass
    try:
        driver.execute_script("document.activeElement && document.activeElement.blur && document.activeElement.blur();")
    except Exception:
        pass
    try:
        driver.execute_script("document.body && document.body.click && document.body.click();")
    except Exception:
        pass

def set_date_input_by_locators(locators, value_ddmmyyyy, attempts=3):
    last_err = None
    for _ in range(attempts):
        try:
            close_overlays()
            el = find_first(locators, timeout_each=10)
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            except Exception:
                pass
            driver.execute_script("""
                const el = arguments[0];
                el.removeAttribute('readonly');
                el.removeAttribute('disabled');
                el.style.display = 'block';
                el.style.visibility = 'visible';
                el.value = '';
            """, el)
            driver.execute_script("""
                const el = arguments[0], val = arguments[1];
                el.value = val;
                el.dispatchEvent(new Event('input', {bubbles:true}));
                el.dispatchEvent(new Event('change', {bubbles:true}));
                el.dispatchEvent(new Event('blur', {bubbles:true}));
            """, el, value_ddmmyyyy)
            time.sleep(0.2)
            return True
        except (StaleElementReferenceException, ElementNotInteractableException) as e:
            last_err = e
            time.sleep(0.3)
            continue
        except Exception as e:
            last_err = e
            time.sleep(0.3)
            continue
    if last_err: raise last_err
    return False

# =========================
# SCRIPT START TIME
# =========================
script_start_time = time.time()

# =========================
# MAIN FLOW
# =========================
frames = []

try:
    print("Opening site…")
    driver.get(BASE_URL)

    # --- Login ---
    print("Logging in…")
    username_field = wait.until(EC.presence_of_element_located((By.ID, "UserName")))
    password_field = wait.until(EC.presence_of_element_located((By.ID, "Password")))
    login_button = wait.until(EC.element_to_be_clickable((By.ID, "sign")))

    driver.execute_script("arguments[0].removeAttribute('readonly');", username_field)
    driver.execute_script("arguments[0].removeAttribute('readonly');", password_field)
    username_field.clear(); username_field.send_keys(USERNAME)
    password_field.clear(); password_field.send_keys(PASSWORD)
    login_button.click()

    wait.until(EC.any_of(
        EC.presence_of_element_located((By.XPATH, "//aside|//div[contains(@class,'sidebar')]")),
        EC.presence_of_element_located((By.XPATH, "//a[contains(.,'Logout') or contains(@href,'Logout')]"))
    ))
    time.sleep(1.0)

    # --- Data Management ---
    print("Opening Data Management…")
    safe_click((
        By.XPATH,
        "//a[contains(@href,'/Admin/DataManagement/Index') and .//span[contains(text(),'Data Management')]]"
    ), timeout=40)

    # --- Store Action ---
    print("Opening Store Action…")
    StoreAction_locators = [
        (By.XPATH, "//a[contains(.,'Store Action') and not(contains(@href,'PrimaryShelfStock'))]"),
        (By.XPATH, "//a[contains(@href,'/Store Action') and contains(.,'Store Action')]"),
    ]
    opened_store_action = False
    for loc in StoreAction_locators:
        try:
            safe_click(loc, timeout=40)
            opened_store_action = True
            break
        except Exception:
            continue
    if not opened_store_action:
        raise TimeoutException("Could not open StoreAction (all locators failed).")

    # --- Primary Shelf Stock ---
    print("Opening Primary Shelf Stock…")
    monthly_locators = [
        (By.XPATH, "//a[contains(@href,'/StoreActionV2/PrimaryShelfStockV2') and contains(.,'Primary Shelf Stock')]"),
        (By.XPATH, "//a[contains(.,'Primary Shelf Stock')]"),
    ]
    opened_primary = False
    for loc in monthly_locators:
        try:
            el = WebDriverWait(driver, 40).until(EC.presence_of_element_located(loc))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            try: el.click()
            except Exception: js_click(el)
            opened_primary = True
            break
        except Exception:
            continue
    if not opened_primary:
        raise TimeoutException("Could not open Primary Shelf Stock (all locators failed).")

    try:
        WebDriverWait(driver, 40).until(lambda d: "StoreAction" in d.current_url)
    except Exception:
        pass

    ensure_in_correct_frame()

    print("Selecting Datewise…")
    try:
        select2_container = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@id='select2-Sdatewise-container']"))
        )
        select2_container.click()
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@class,'select2-search__field')]"))
        )
        search_box.clear(); search_box.send_keys("Datewise"); time.sleep(0.3); search_box.send_keys(Keys.ENTER)
    except TimeoutException:
        try:
            datewise_option = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[contains(@id,'select2-Sdatewise') and normalize-space()='Datewise']"))
            )
            js_click(datewise_option)
        except Exception as e:
            print("Select2 fallback failed:", e)

    FROM_LOCATORS = [
        (By.ID, "Sdate"),
        (By.NAME, "DateFrom"),
        (By.XPATH, "//input[@placeholder='From' or @placeholder='Date From']")
    ]
    TO_LOCATORS = [
        (By.ID, "dateto"),
        (By.NAME, "DateTo"),
        (By.XPATH, "//input[@placeholder='To' or @placeholder='Date To']")
    ]

    first_day_dt = datetime.today().replace(day=1)
    yesterday_dt = datetime.today() - timedelta(days=1)
    print(f"Block iteration: {first_day_dt.strftime('%d/%m/%Y')} -> {yesterday_dt.strftime('%d/%m/%Y')} in {BLOCK_DAYS}-day blocks")

    block_start = first_day_dt
    block_index = 1

    while block_start <= yesterday_dt:
        block_end = min(block_start + timedelta(days=BLOCK_DAYS - 1), yesterday_dt)
        from_str = block_start.strftime("%d/%m/%Y")
        to_str   = block_end.strftime("%d/%m/%Y")
        print(f"\n=== Block {block_index}: {from_str} → {to_str} ===")

        try:
            ensure_in_correct_frame()
            close_overlays()

            # Set From/To for the whole block
            set_date_input_by_locators(FROM_LOCATORS, from_str, attempts=4)
            set_date_input_by_locators(TO_LOCATORS,   to_str,   attempts=4)

            # Open "Download Now"
            download_dropdown_locators = [
                (By.XPATH, "//a[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
                (By.XPATH, "//button[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
            ]
            opened = False
            for loc in download_dropdown_locators:
                try:
                    safe_click(loc, attempts=4)
                    opened = True
                    break
                except Exception:
                    continue
            if not opened:
                print("⚠️ Could not open 'Download Now' dropdown. Skipping this block.")
                block_start = block_end + timedelta(days=1); block_index += 1
                continue

            # Click Export
            master_export_locators = [
                (By.XPATH, "//button[@type='submit' and contains(@class,'dropdown-item') and @value='Export' and contains(.,'Export')]"),
                (By.XPATH, "//button[contains(.,'Export') and (@type='submit' or @value='Export')]"),
                (By.XPATH, "//a[contains(.,'Export')]"),
            ]
            exported = False
            for loc in master_export_locators:
                try:
                    safe_click(loc, attempts=4)
                    exported = True
                    break
                except Exception:
                    continue
            if not exported:
                print("⚠️ Could not click Export. Skipping this block.")
                block_start = block_end + timedelta(days=1); block_index += 1
                continue

            # Wait for file
            downloaded_path = wait_for_download(DOWNLOAD_DIR, timeout=240)
            if not downloaded_path:
                print(f"ℹ️ No file downloaded for {from_str}→{to_str}. Skipping.")
                block_start = block_end + timedelta(days=1); block_index += 1
                continue

            try:
                if os.path.getsize(downloaded_path) < 5:
                    print(f"ℹ️ Empty file for {from_str}→{to_str}. Skipping.")
                    os.remove(downloaded_path)
                    block_start = block_end + timedelta(days=1); block_index += 1
                    continue
            except Exception:
                pass

            df = parse_table(downloaded_path)
            if df is None or df.empty:
                print(f"ℹ️ No data inside file for {from_str}→{to_str}. Skipping.")
                try: os.remove(downloaded_path)
                except Exception: pass
                block_start = block_end + timedelta(days=1); block_index += 1
                continue

            # Tag the pull range for traceability
            df.insert(0, "PulledFrom", block_start.strftime("%Y-%m-%d"))
            df.insert(1, "PulledTo",   block_end.strftime("%Y-%m-%d"))
            frames.append(df)
            print(f"✅ Loaded {len(df)} rows for block {from_str}→{to_str}")

            try:
                os.remove(downloaded_path)
            except Exception as e:
                print(f"⚠️ Could not delete temp file {downloaded_path}: {e}")

            time.sleep(0.6)

        except (StaleElementReferenceException, ElementNotInteractableException) as e:
            print(f"⚠️ Interactability/Stale issue on block {from_str}→{to_str}: {e}. Skipping this block.")
        except Exception as e:
            print(f"⚠️ Unexpected error on block {from_str}→{to_str}: {e}. Skipping this block.")

        block_start = block_end + timedelta(days=1)
        block_index += 1

    # =========================
    # WRITE ONE FINAL CSV
    # =========================
    if frames:
        compiled = pd.concat(frames, ignore_index=True)
        compiled_name = f"PrimaryShelfStock_compiled_{first_day_dt.strftime('%Y%m%d')}_{yesterday_dt.strftime('%Y%m%d')}_block{BLOCK_DAYS}.csv"
        compiled_path = os.path.join(DOWNLOAD_DIR, compiled_name)
        compiled.to_csv(compiled_path, index=False, encoding="utf-8-sig")
        print(f"📦 Compiled CSV created: {compiled_path} (rows: {len(compiled)})")
    else:
        print("⚠️ No readable files to compile. Skipped creating compiled CSV.")

except Exception as e:
    print("❌ Automation failed:", e)
finally:
    try:
        driver.quit()
    except Exception:
        pass
    script_end_time = time.time()
    total_runtime = script_end_time - script_start_time
    minutes, seconds = divmod(total_runtime, 60)
    print(f"⏱ Total script runtime: {int(minutes)} min {int(seconds)} sec")


Deleted: C:\Users\Administrator\Rahul\downloads\downloads\Monthly_Attendance_20251101_20251105_block10.csv
Opening site…
Logging in…
Opening Data Management…
Opening Store Action…
Opening Primary Shelf Stock…
Selecting Datewise…
Block iteration: 01/11/2025 -> 05/11/2025 in 10-day blocks

=== Block 1: 01/11/2025 → 05/11/2025 ===
⏱ Total script runtime: 13 min 34 sec


KeyboardInterrupt: 

In [40]:
# SOSData.to_csv(r'C:\Users\Administrator\LIS\Dabur\DaburSOSData.csv', index=False)

# specify folder path
folder_path = r'C:\Users\Administrator\Rahul\downloads\downloads'

csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

print(csv_files)  # just to check what files were found

[]


In [41]:
# emailer

from datetime import datetime, timedelta
import smtplib
from email.message import EmailMessage
import mimetypes
import os
import zipfile
import glob

# Email setup
sender_email   = "reports.mis@loveinstore.com"
app_password   = "ydfd ntvg qcfp xxbi"
recipients     = ["kunalk@loveinstore.com", "rahuly@loveinstore.com", "pankajs@loveinstore.com"]

subject_tpl    = "Dabur SOS CSV Dump - {date} {time}"
body_template  = """
Please find attached Data for {date} at {time}

Regards,
Team Love In Store
"""

# Date setup
now = datetime.now() - timedelta(days=1)
rounded = now.replace(minute=0, second=0, microsecond=0)
time_str = rounded.strftime("%I:%M %p")
formatted_date = now.strftime("%d-%b-%Y")

# Define folder and new ZIP file path
zip_folder = r"C:\Users\Administrator\Rahul\downloads\downloads\Zip_file"
zip_file = os.path.join(zip_folder, f"Reports_{formatted_date.replace('-', '')}.zip")

# === NEW FEATURE: Delete old ZIP files before creating new one ===
for old_zip in glob.glob(os.path.join(zip_folder, "*.zip")):
    try:
        os.remove(old_zip)
        print(f"Deleted old ZIP file: {old_zip}")
    except Exception as e:
        print(f"Error deleting {old_zip}: {e}")

# === Create new ZIP file ===
# Make sure you define 'csv_files' somewhere above (list of CSV file paths)
csv_files = [
    # Example:
    # r"C:\path\to\file1.csv",
    # r"C:\path\to\file2.csv"
]

with zipfile.ZipFile(zip_file, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file_path in csv_files:
        zipf.write(file_path, os.path.basename(file_path))  # store only filename in ZIP
        print(f"Added: {file_path}")

# === Create and send email ===
msg = EmailMessage()
msg["From"]    = sender_email
msg["To"]      = ", ".join(recipients)
msg["Subject"] = subject_tpl.format(date=formatted_date, time=time_str)
msg.set_content(body_template.format(date=formatted_date, time=time_str))

# Attach the ZIP
with open(zip_file, "rb") as f:
    file_data = f.read()
msg.add_attachment(file_data, maintype="application", subtype="zip", filename=os.path.basename(zip_file))

# Send email
with smtplib.SMTP("smtp.gmail.com", 587) as smtp:
    smtp.starttls()
    smtp.login(sender_email, app_password)
    smtp.send_message(msg)

print("Email sent successfully with the new ZIP file.")


Email sent successfully with the new ZIP file.
